<a href="https://colab.research.google.com/github/nogbazghi/CSC8980/blob/main/Exam_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nahom Ogbazghi
002052292

Question 1) (20 points) Write a function that takes a List of five words: [‘apple’, ‘house’, ‘pear’,
‘dog’, ‘doctor’] and returns a list of lists with each element being a word and a list of the top five
most similar words. For this task you have to use the most suitable method of the ones we have
seen in class to determine the most similar words to the original input list. You can use a
pre-trained resource if you think is appropriate. After calling your function, print the most similar
words to the screen. Are these ‘similar’ words actually similar? If not, why not? What do you
think can be improved and how - talk about it, do not necessarily implement it?


In [2]:
randomSeed = 12345

In [3]:
!gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM

Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
1.65GB [00:11, 147MB/s]


In [4]:
!gunzip /content/GoogleNews-vectors-negative300.bin.gz

gzip: /content/GoogleNews-vectors-negative300.bin already exists; do you wish to overwrite (y or n)? ^C


In [5]:
from gensim.models import KeyedVectors
filename = '/content/GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [6]:
print("Words and there five most similar words")
words = ['apple', 'house', 'pear', 'dog', 'doctor']
def mostSimilarWords(wordsList):
  allSimilarWords = []
  for word in wordsList:
    similarWords = []
    similarWords = model.most_similar(word)
    topFIve = [wordTuple[0] for wordTuple in similarWords[:5]]
    allSimilarWords.append([word, topFIve])
  return allSimilarWords
mostSimilarWords(words)

Words and there five most similar words


[['apple', ['apples', 'pear', 'fruit', 'berry', 'pears']],
 ['house', ['houses', 'bungalow', 'apartment', 'bedroom', 'townhouse']],
 ['pear', ['pears', 'apricot', 'apricots', 'nectarine', 'Fuji_apple']],
 ['dog', ['dogs', 'puppy', 'pit_bull', 'pooch', 'cat']],
 ['doctor', ['physician', 'doctors', 'gynecologist', 'surgeon', 'dentist']]]

**ANSWER:**
All in all the majority were similar, but there occasionally were words that could have ben substituted for a more similar word: (condo - instead of bedroom),(another type of dog- instead of cat), and really all words for apple except for apples. Improvements can be made by considering a more unique corpus to search each word through versus a general Google one. A fruit corpus for apple/pears, medical corpus for doctor, etc. This would allow for better contextualized information.


Question 2) (30 points) Using the Homework 2 dataset, also attached in the Exam 2 files,
shakespeares-works_TXT_FolgerShakespeare.zip. Find the document to document similarity
using:
a) Cosine similarity. And create a 42 x 42 heatmap of these similarities.
b) Use Doc2Vec to create document embeddings and find the similarities between the
documents. To visualize this, also create a 42 x 42 heatmap for this.
c) What are the differences you find between the two methods? Is there anything radically
different? Please describe your answer in terms of the heatmap of part a and part b.


In [7]:
# !pip install wget
# import wget
!mkdir -p data
!unzip -n -d data /content/shakespeares-works_TXT_FolgerShakespeare.zip

Archive:  /content/shakespeares-works_TXT_FolgerShakespeare.zip


In [8]:
import nltk
nltk.download('reuters')
nltk.download('punkt')

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
import os
import re
import math
import sklearn.metrics
import nltk
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

dir = "/content/data/"
matrix = {}
N=42
getWord = "[0-9.!:(@)?\\-\*\#]*([a-zA-Z]+\'?[a-z]*)[0-9.(@:)!?\\-\*\#]*"
compile = re.compile(getWord)
allFilenames = []
taggedDocs = []
shakespearContent = []
for filename in os.listdir(dir):
  if filename.endswith(".txt"):
    allFilenames.append(filename)
    doctwovecContent = []
    wordcount_for_File = {}
    content = open(dir+str(filename), 'r').read()
    shakespearContent.append(content)
    words = content.split()
    for raw_word in words:
      w = raw_word.lower()
      match = compile.match(w)
      if match:
        grabbedWord = match.group(1)
        if grabbedWord not in doctwovecContent:
          doctwovecContent.append(grabbedWord)
        if grabbedWord in matrix:
          grabbedWord_dict = matrix[grabbedWord]
          if filename in grabbedWord_dict:
            wordCount = grabbedWord_dict[filename]
            wordCount+=1
            grabbedWord_dict[filename] = wordCount
            matrix[grabbedWord] = grabbedWord_dict
          else:
            grabbedWord_dict[filename] = 1
            matrix[grabbedWord] = grabbedWord_dict
        else:
          matrix[grabbedWord] = {filename: 1}
    #doc2vec api recommended passing a taggedDocument object
    taggedDoc = TaggedDocument(words=doctwovecContent, tags=[filename])
    taggedDocs.append(taggedDoc)

# Structure of matrix
# {word : {filename : wordCount within file}}

In [10]:
def tf(word, doc):
  if word in matrix:
    files = matrix[word]
    if doc in files:
      count = files[doc]
      return count
  else:
    return 0

def idf(word):
  docCount = len(matrix[word])
  # print(docCount)
  idf = N/docCount
  return idf

def tf_idf(word, doc):
  tfValue = tf(word, doc)
  idfValue = idf(word)
  return tfValue * idfValue

In [11]:
all_words = matrix.keys()
tfidfsMatrix = {}
for word in all_words:
  listOfFileCount = matrix[word]
  for f in listOfFileCount:
    tf_idfCount = tf_idf(word, f)
    if f in tfidfsMatrix:
      word_tfidf = tfidfsMatrix[f]
      word_tfidf[word] = tf_idfCount
    else:
      word_tfidf = {word: tf_idfCount}
      tfidfsMatrix[f] = word_tfidf

In [12]:
def denomCS(fileA, fileB):
  fileAwks = list(tfidfsMatrix[fileA].values())
  fileBwks = list(tfidfsMatrix[fileB].values())
  kA = len(fileAwks)
  kB = len(fileBwks)
  totalA = 0
  for tfidfScore in fileAwks:
    totalA += math.pow(tfidfScore, 2)
  totalB=0
  for tfidfScore in fileBwks:
    totalB += math.pow(tfidfScore, 2)
  left = math.sqrt(totalA)
  right = math.sqrt(totalB)
  denom = left * right
  return denom

In [13]:
tfidfsScoreMatrix = []
for keyfile in allFilenames:
  scores = []
  for secondFile in allFilenames:
    keywords = tfidfsMatrix[keyfile]
    secondwords = tfidfsMatrix[secondFile]
    denom = denomCS(keyfile, secondFile)
    num = 0
    wordsToUse = []
    if len(keywords) < len(secondwords):
      wordsToUse = keywords
    else:
      wordsToUse = secondwords
    for word in wordsToUse:
      if word in keywords and word in secondwords:
        keyVal = keywords[word]
        secVal = secondwords[word]
        mult = keyVal * secVal
        num += mult
    score = num/denom
    scores.append(score)
  tfidfsScoreMatrix.append(scores)

In [14]:
from sklearn.metrics import confusion_matrix

print(tfidfsScoreMatrix)
# mat = confusion_matrix(tfidfsScoreMatrix, allFilenames)
# sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
#             xticklabels=allFilenames, yticklabels=allFilenames)
# plt.xlabel('true label')
# plt.ylabel('predicted label');

[[0.9999999999999999, 0.03883830101249319, 0.026366467716441393, 0.02833673470500418, 0.062195309806368164, 0.05210219869914298, 0.06520495422556485, 0.028641156821921893, 0.07057722591429658, 0.02412903121161592, 0.02603132188818433, 0.02317543341239399, 0.013170215879546367, 0.09908250079153197, 0.020592088944247612, 0.0209477147254021, 0.033626515059821935, 0.03151665724059068, 0.01596295805813951, 0.06653009920357413, 0.029231806947803876, 0.11077835032100461, 0.031329829128391565, 0.022744979956505824, 0.023542315976830996, 0.017292064995466812, 0.08597239301963992, 0.05914099490369381, 0.07583524141781998, 0.026183580316338533, 0.0242634863433639, 0.03398716865458982, 0.03287429242156608, 0.1416974872252486, 0.019997642422751528, 0.09095400592124175, 0.04403965905185058, 0.027057096929003883, 0.025831064053697617, 0.021416162760757158, 0.07413242719920399, 0.030798225359168065], [0.03883830101249319, 1.0, 0.031773452777253484, 0.03539881766662724, 0.07637670102039307, 0.065438025

In [15]:
model = Doc2Vec(taggedDocs, vector_size=42,  min_count=1, size=300, workers=1, window=5, max_vocab_size=2000, seed=randomSeed)

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [16]:
allscores = []
for file in allFilenames:
  docVector = model.infer_vector(taggedDoc[file])
  mostSim=model.dv.most_similar(positive=[docVector])
  dictionarySim = dict(mostSim) 
  scores = []
  for f in allFilenames:
    if f not in dictionarySim:
      scores.append(0)
    else:

      scores.append(dictionarySim[f])
  allscores.append(scores)

TypeError: ignored

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

print(allscores)

# mat = multilabel_confusion_matrix(allscores, allscores)
# sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
#             xticklabels=allFilenames, yticklabels=allFilenames)
# plt.xlabel('true label')
# plt.ylabel('predicted label');

**ANSWER:**
The heatmap was not able to be created but the score were printed out manually. As wellas doc2vec did not compile. Based on an assumption, I would say doc2vec would have preformed better simply because most of the calculations in the first implementation did not score higher than 0.1% except for when a filename is matched with itself. If my assumption that doc2vec performs better I believe this would have everything to do with tokenization as the implementations to score the words should not have any impact. The tokens may have had the largest deviation in quality that impacted the scores.

Question 3) (30 points) Using the Homework 2 dataset. Use SpaCy to extract the following:

In [17]:
import spacy
from nltk import bigrams, trigrams
nlp = spacy.load("en_core_web_sm")

In [18]:
from collections import Counter, defaultdict


b) Write a function to generate all unique trigrams from all documents in the dataset. The
input of this function should be the concatenated dataset and the output should be the
list of trigrams and their frequency. Display the top 10 most common trigrams and their
frequency

In [19]:
def buildTrimodel(all_content):
  trimodel = defaultdict(lambda: defaultdict(lambda: 0))
  for content in all_content:
    sentences = nltk.sent_tokenize(content)
    for sentence in sentences:
      doc = nlp(sentence)
      for w1, w2, w3 in trigrams(doc, pad_right=True, pad_left=True):
        trimodel[(w1, w2)][w3] += 1
  return trimodel

In [20]:
def getSmallestItem(items):
  keys = list(items.keys())
  smallestVal = items[keys[0]]
  smallestKey = keys[0]
  for key in keys:
    val = items[key]
    if val < smallestVal:
      smallestVal = val
      smallestKey = key
  return (smallestKey, smallestVal)

In [21]:
def buildTrigram(all_content):
  trimodel = buildTrimodel(all_content)
  tri_ten = {}
  for w1_w2 in trimodel:
    total_count = float(sum(trimodel[w1_w2].values()))
    #fill it up
    if len(tri_ten) < 10:
      tri_ten[w1_w2] = total_count
    else:
      # replace smallest if large enough
      smallestKey, smallestVal = getSmallestItem(tri_ten)
      if total_count > smallestVal:
        tri_ten.pop(smallestKey)
        tri_ten[w1_w2] = total_count
  return tri_ten

In [22]:
print(buildTrigram(shakespearContent))

{(None, None): 78295.0, (None, The): 1.0, (The, Winter): 1.0, (Winter, 's): 1.0, ('s, Tale): 1.0, (Tale, 
): 1.0, (
, by): 1.0, (by, William): 1.0, (William, Shakespeare): 1.0, (Shakespeare, 
): 1.0}


a) Write a function to generate all unique bigrams from all documents in the dataset. The
input of this function should be the concatenated dataset and the output should be the
list of bigrams and their frequency. Display the top 10 most common bigrams and their
frequency.

In [23]:
def buildBimodel(all_content):
  bimodel = defaultdict(lambda: defaultdict(lambda: 0))
  for content in all_content:
    sentences = nltk.sent_tokenize(content)
    for sentence in sentences:
      doc = nlp(sentence)
      for w1, w2 in bigrams(doc, pad_right=True, pad_left=True):
        bimodel[w1][w2] += 1
  return bimodel

In [24]:
def buildBigram(all_content):
  bimodel = buildBimodel(all_content)
  bi_ten = {}
  for w1 in bimodel:
    total_count = float(sum(bimodel[w1].values()))
    #fill it up
    if len(bi_ten) < 10:
      bi_ten[w1] = total_count
    else:
      # replace smallest if large enough
      smallestKey, smallestVal = getSmallestItem(bi_ten)
      if total_count > smallestVal:
        bi_ten.pop(smallestKey)
        bi_ten[w1] = total_count
  return bi_ten

In [ ]:
print(buildBigram(shakespearContent))

c) Write a function to extract all unique NOUN and VERB tokens. The input of this function
should be the concatenated dataset and the output should be two lists: one of the NOUN
tokens and their frequency, the other list should be the VERB tokens and their counts.
Display the top 10 most common NOUN and VERB tokens.

In [ ]:
def posFinder(all_content):
  nounCounter = {}
  verbCounter = {}
  ntop_ten = {}
  vtop_ten = {}
  for content in all_content[:3]:
    sentences = nltk.sent_tokenize(content)
    for sentence in sentences:
      doc = nlp(sentence)
      for t in doc:
        pos = t.pos_
        
        word = t.text.strip().lower()
        if pos == "NOUN":
          if word in nounCounter:
            nounCounter[word] += 1
          else:
            nounCounter[word] = 1
        elif pos == "VERB":
          if word in verbCounter:
            verbCounter[word] += 1
          else:
            verbCounter[word] = 1
  nkeys = nounCounter.keys()
  vkeys = verbCounter.keys()
  for key in nkeys:
    if len(ntop_ten) < 10:
      ntop_ten[key] = nounCounter[key]
    else:
      smallestKey, smallestVal = getSmallestItem(ntop_ten)
      count = nounCounter[key]
      if count > smallestVal:
        ntop_ten.pop(smallestKey)
        ntop_ten[key] = count
  vkeys = verbCounter.keys()
  for key in vkeys:
    if len(vtop_ten) < 10:
      vtop_ten[key] = verbCounter[key]
    else:
      smallestKey, smallestVal = getSmallestItem(vtop_ten)
      count = verbCounter[key]
      if count > smallestVal:
        vtop_ten.pop(smallestKey)
        vtop_ten[key] = count
  return(ntop_ten, vtop_ten)

In [ ]:
posFinder(shakespearContent)

d) What do you think the most common bigrams and trigrams could be useful for? There is
a particular method we have seen in this class to characterize a corpus that could benefit
from having these bigrams/trigrams when the underlying text corpus can’t be shared.
Please talk about this.

It would be useful for mapping words to other concepts/clustering, showing relationships between corpuses but not actually exposing the corpuses. Providing a perspective on the data from its key words.

Question 4) (30 points) Using the dataset: Ask0729, found in Exam files, write two functions to
extract all dates found in this dataset. The input of these functions should take the dataset as
input, and output a list of dates. You should use two different methods, one per function.

In [ ]:
!mkdir -p ask
!unzip -n -d ask "/content/Ask0729.zip"

a) First method: using SpaCy (this is a big enough hint)

In [ ]:
ask_content = open(str("/content/ask/Ask0729-fixed.txt"), 'r').read()

In [ ]:
# months
months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

In [ ]:
def build_date(one, two, three):
  if len(three) == 0:
    return one+" "+two
  return one+" "+two+" "+three

In [ ]:
#possible combos: M D Y, M Y, D M Y
def getDatesUsingSpaCy(content):
  spacy_dates = []
  second = ["", ""]
  first = ["", ""]
  for word in content.split():
    doc = nlp(word)
    for t in doc:
      row = [t.text, t.pos_]
      if first[1]  == "NUM": #starts with day
        if second[1] == "PROPN":
          # make sure it is a month
          lower_second = second[0].lower()
          for m in months:
            if lower_second.startswith(m):
              if row[1] == "NUM":
                spacy_dates.append(build_date(first[0], second[0], row[0]))
              else:
                spacy_dates.append(build_date(first[0], second[0], ""))
      elif first[1]  == "PROPN": #starts with month
        lower_first = first[0].lower()
        for m in months:
          if lower_second.startswith(m):
            if second[1] == "NUM":
              if row[1] == "NUM":
                spacy_dates.append(build_date(first[0], second[0], row[0]))
              else:
                spacy_dates.append(build_date(first[0], second[0], ""))
      first = second
      second = row
  return spacy_dates

In [ ]:
print(getDatesUsingSpaCy(ask_content))

b) Second method: using regular expressions.

In [ ]:
import re
def getDatesUsingRegex(content):
  day_pattern = "\w*\s?([0-9]|0[1-9]|[1-2][0-8]|29|3[0-1])\s?\w*"
  month_pattern = "\w*\s?(jan|january|feb|february|mar|march|apr|april|may|jun|june|jul|july|aug|august|sep|september|oct|october|nov|november|dec|december)\s?\w*"
  year_pattern = "\w*\s?(18[0-9][0-9]|19[0-9][0-9]|200[0-9])\s?\w*"
  compileDay = re.compile(day_pattern)
  compileMonth = re.compile(month_pattern)
  compileYear = re.compile(year_pattern)
  dates = []
  buildDate = ""
  for raw_word in content.split():
    word = raw_word.lower().strip()
    matchDay = compileDay.match(raw_word)
    matchMonth = compileMonth.match(raw_word)
    matchYear = compileYear.match(raw_word)
    if matchDay:
      buildDate += word
    elif matchMonth:
      buildDate += word
    elif matchYear:
      buildDate += word
  dates.append(buildDate)
  buildDate = ""
  return dates

In [ ]:
print(getDatesUsingRegex(ask_content))

**ANSWER:**

The better approach was the SpaCy, because the parsing is much more organized and has better identifiers, such as Proper nouns and num. That allowed me to narrow down my list to search for. REgex did not produce what was necessary as the pattern became too complex. I would use both combined, and not purely one or the other. Sometimes the indicators of what a date would produce false positives like Houston 2007. Regex would then allow for me to filter that out.

Question 5) (30 points) Train an LSTM model to classify the Cornell Movie Review data
using the polarity_dataset V2.0. You can use the code for class 19, but take a note that
you will have to adapt some of the parameters like: Review size = 450, epochs=5. You will use
85% of the dataset for training, and 15% for testing. Once you build the model, please display
the sklearn classification report. What are you noticing here? Anything unexpected? How does
this model compare to the one built with the IMDB dataset in class? Any ideas on how to
improve it?


In [ ]:
!pip install wget

In [ ]:
!mkdir -p lsmt
!wget -nc http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz /content/
# !gunzip /content/review_polarity.tar.gz
!tar -xvzf /content/review_polarity.tar.gz -C /content/lsmt

In [ ]:
import pandas as pd
# Supress deprecation warnings
import logging
logging.getLogger('tensorflow').disabled = True

In [ ]:
import pandas as pd

pos_df = []
neg_df = []
pos_num = []
neg_num = []
class_names=["negative", "positive"]
polarity = ["neg/", "pos/"]
path = "/content/lsmt/txt_sentoken/"
for p in polarity:
  for filename in os.listdir(path+p):
    filepath = path+p+filename
    file = open(filepath, 'r')
    content = file.read()
    split_content = content.split()
    if(p == "pos/"):
      pos_df.append(content)
      pos_num.append(1)
    else:
      neg_df.append(content)
      neg_num.append(0)

positivePanda = pd.DataFrame({"polarity":pos_num, "text":pos_df})
negativePanda = pd.DataFrame({"polarity":neg_num, "text":neg_df})
frames = [positivePanda, negativePanda]
df = pd.concat(frames)

In [ ]:
from sklearn.utils import shuffle
df = shuffle(df)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vocab_size=10000
vectorizer = TfidfVectorizer(max_features=vocab_size)
vectorizer = TfidfVectorizer(max_features=vocab_size)
vectors = vectorizer.fit_transform(df.text)
word_id_pair = vectorizer.vocabulary_
words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
words_df.head()
X = words_df
y = df.polarity

In [ ]:
from sklearn.model_selection import train_test_split

A_X_train, A_X_test, A_y_train, A_y_test = train_test_split(X, y, train_size=0.85, random_state=12345)

In [ ]:
# All the imports!
import tensorflow as tf 
import numpy as np
from tensorflow.keras.preprocessing import sequence
from numpy import array
import re
import random

randomSeed = 12345
tf.random.set_seed(randomSeed)
random.seed(randomSeed)
np.random.seed(randomSeed)

In [ ]:
# need to extract words from difficult text
getWord = "[0-9.!:(@)?\\-\*\#]*([a-zA-Z]+)[0-9.(@:)!?\\-\*\#]*"
compile = re.compile(getWord)

#convert text to ints
intTweets = []
for tweet in df.text:
  split = tweet.split(" ")
  wordInt = []
  for raw_word in split:
    word = raw_word.lower()
    if word not in word_id_pair:
      match = compile.match(word)
      if match:
        grabbedWord = match.group(1)
        if grabbedWord in word_id_pair:
          wordInt.append(word_id_pair[grabbedWord])
        else:
          randomValForWord = random.random()
          word_id_pair[word] = randomValForWord
          wordInt.append(randomValForWord)
    else:
      wordInt.append(word_id_pair[word])
  intTweets.append(wordInt)

In [ ]:
#need to provide uniform length
#preset to 450
review_length = 450

# Padding / truncated our reviews
intTweets_padded = sequence.pad_sequences(intTweets, maxlen = review_length)

# split the tweet using the same randomseed
# a 85% training, 15 test
tweets_train, tweets_test, y_train, y_test = train_test_split(intTweets_padded, y, train_size=0.85, random_state=randomSeed)

In [ ]:
# We begin by defining the a empty stack. We'll use this for building our 
# network, later by layer.
model = tf.keras.models.Sequential()

# The Embedding Layer provides a spatial mapping (or Word Embedding) of all the 
# individual words in our training set. Words close to one another share context 
# and or meaning. This spatial mapping is learning during the training process.
model.add(
    tf.keras.layers.Embedding(
        input_dim = vocab_size, # The size of our vocabulary 
        output_dim = 32, # Dimensions to which each words shall be mapped
        input_length = review_length # Length of input sequences
    )
)

# Dropout layers fight overfitting and forces the model to learn multiple 
# representations of the same data by randomly disabling neurons in the 
# learning phase.
model.add(
    tf.keras.layers.Dropout(
        rate=0.25 # Randomly disable 25% of neurons
    )
)

# We are using a fast version of LSTM whih is optimised for GPUs. This layer 
# looks at the sequence of words in the review, along with their word embeddings
# and uses both of these to determine to sentiment of a given review.
model.add(
    tf.keras.layers.LSTM(
        units=32 # 32 LSTM units in this layer
    )
)

# Add a second dropout layer with the same aim as the first.
model.add(
    tf.keras.layers.Dropout(
        rate=0.25 # Randomly disable 25% of neurons
    )
)

# All LSTM units are connected to a single node in the dense layer. A sigmoid 
# activation function determines the output from this node - a value 
# between 0 and 1. Closer to 0 indicates a negative review. Closer to 1 
# indicates a positive review.
model.add(
    tf.keras.layers.Dense(
        units=1, # Single unit
        activation='sigmoid' # Sigmoid activation function (output from 0 to 1)
    )
)

# Compile the model
model.compile(
    loss=tf.keras.losses.binary_crossentropy, # loss function
    optimizer=tf.keras.optimizers.Adam(), # optimiser function
    metrics=['accuracy']) # reporting metric

# Display a summary of the models structure
model.summary()

In [ ]:
# Train the LSTM on the training data
history = model.fit(

    # Training data : features (review) and classes (positive or negative)
    tweets_train, y_train,
                    
    # Number of samples to work through before updating the 
    # internal model parameters via back propagation. The 
    # higher the batch, the more memory you need.
    batch_size=128, 

    # An epoch is an iteration over the entire training data.
    epochs=5, 
    
    # The model will set apart his fraction of the training 
    # data, will not train on it, and will evaluate the loss
    # and any model metrics on this data at the end of 
    # each epoch.
    validation_split=0.2,
    
    verbose=1
) 

In [ ]:
# Get Model Predictions for test data
from sklearn.metrics import classification_report
predicted_classes = model.predict_classes(tweets_test)
classRep = (classification_report(y_test, predicted_classes, target_names=class_names))
print(classRep)
reportArray = (classRep.split())
precision = (float(reportArray[19])*100.0)
recall = (float(reportArray[20])*100.0)
fOne = (float(reportArray[21])*100.0)
lsmt_scores = ["LSMT", precision, recall, fOne]

**ANSWER:**


I noticed a steep drop in precision relative to recall for negative and a steeper drop from recall relative to precision for prositive. It is much less consistent than the IMDB one as that one remained in the high 80s across all scores. This can be improved by better tokenizing the dataset using a corpus that understands the corpus for better context.

Question 6) (30 points) Use the train.txt file from the PubMed 20K RCT dataset fine-tune a
BERT transformer (class 9 code). This task is a bit different as the one seen in class, here the
source dataset has FIVE different classes: background, objective, method, result, and
conclusion. Once the BERT model is fine-tuned, classify the: test.txt set. Please present the
per-class classification report (accuracy, precision, recall, f1-score metrics). Also, present the
global metrics - all classes (accuracy, precision, recall, f1-score metrics). Did you model beat the
baseline results (https://arxiv.org/pdf/1710.06071.pdf)? What do you think you can do to improve
it?


In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
# else:
#     raise SystemError('GPU device not found')

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
!pip install transformers

In [ ]:
!wget -nc https://github.com/Franck-Dernoncourt/pubmed-rct/raw/master/PubMed_20k_RCT/train.txt /content/

In [ ]:
import pandas as pd

labels = ["BACKGROUND", "OBJECTIVE", "METHODS", "RESULTS", "CONCLUSIONS"]
# Load the dataset into a pandas dataframe.
df = pd.read_csv("/content/train.txt", delimiter='\t', header=None, names=['label', 'sentence'])

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

df['label'].replace({"BACKGROUND": labels.index("BACKGROUND")}, inplace=True)
df['label'].replace({"OBJECTIVE": labels.index("OBJECTIVE")}, inplace=True)
df['label'].replace({"METHODS": labels.index("METHODS")}, inplace=True)
df['label'].replace({"RESULTS": labels.index("RESULTS")}, inplace=True)
df['label'].replace({"CONCLUSIONS": labels.index("CONCLUSIONS")}, inplace=True)
df = df.dropna()
# Display 10 random rows from the data.
df.sample(10)

In [ ]:
# Get the lists of sentences and their labels.
sentences = df.sentence.values
labels = list(df.label.values)

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, truncation = True, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
print(labels)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)
dataset = dataset[:8000]
# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

BertForSequnceCalssificaction consistently failed and I was not able to train and test the model 

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 5, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = randomSeed

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

Evaluate TEST

In [ ]:
!wget -nc https://github.com/Franck-Dernoncourt/pubmed-rct/raw/master/PubMed_20k_RCT/test.txt /content/

In [ ]:
import pandas as pd

labels = ["BACKGROUND", "OBJECTIVE", "METHODS", "RESULTS", "CONCLUSIONS"]
# Load the dataset into a pandas dataframe.
df = pd.read_csv("/content/test.txt", delimiter='\t', header=None, names=['label', 'sentence'])

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

df['label'].replace({"BACKGROUND": labels.index("BACKGROUND")}, inplace=True)
df['label'].replace({"OBJECTIVE": labels.index("OBJECTIVE")}, inplace=True)
df['label'].replace({"METHODS": labels.index("METHODS")}, inplace=True)
df['label'].replace({"RESULTS": labels.index("RESULTS")}, inplace=True)
df['label'].replace({"CONCLUSIONS": labels.index("CONCLUSIONS")}, inplace=True)
df = df.dropna()
# Display 10 random rows from the data.
df.sample(10)

In [ ]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, truncation = True, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])


In [ ]:
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)